In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import pexpect
import html
import re
from IPython.display import display
import json
from matplotlib import pyplot as plt
import ast
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
from google.colab.drive import mount
import pickle
import urllib.request
import os
import gzip
import shutil

In [ ]:
!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 827.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: Operation cancelled by user


In [ ]:
mount('./data')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Drive already mounted at ./data; to attempt to forcibly remount, call drive.mount("./data", force_remount=True).


In [ ]:
df = pd.read_json('./cellphones_review.jsonl', lines=True, nrows=10)
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600
5,2,True,"01 27, 2014",A1DW2L6XCC5TJS,7508492919,NaN,Amazon Customer,The product looked exactly like the picture an...,Not so happy,1390780800
6,3,True,"01 23, 2014",AQC61R4UST7UH,7508492919,NaN,DaMara Estep,I FINALLY got my case today. It took forever t...,It's cute!,1390435200
7,5,True,"01 17, 2014",A31OVFL91BCKXG,7508492919,NaN,Ashley Nicole Miller,It is a very cute case. None of the jewels hav...,Cute case,1389916800
8,1,True,"12 27, 2013",A1K0VLK6O5Z22M,7508492919,NaN,BeeLove21,DO NOT BUY! this item is seriously cheap as he...,WORST ITEM!,1388102400
9,4,True,"12 16, 2013",A1K3BWU73YB44P,7508492919,NaN,Mrs. Ochoa,I really love this case... you have to keep yo...,Pretty Cute!,1387152000


In [ ]:
df.iloc[5].description

['A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable).']

In [ ]:
  ID of the reviewer, e.g. A2SUAM1J3GNN3B
  ID of the product, e.g. 0000013714
  name of the reviewer
  helpful votes of the review
  a disctionary of the product metadata, e.g., "Format" is "Hardcover"
  text of the review
  rating of the product
  summary of the review
  time of the review (unix time)
  time of the review (raw)
  images that users post after they have received the product

Index(['also_buy', 'also_view', 'asin', 'brand', 'category', 'date',
       'description', 'details', 'feature', 'fit', 'imageURL',
       'imageURLHighRes', 'main_cat', 'price', 'rank', 'similar_item', 'tech1',
       'tech2', 'title'],
      dtype='object')

## Configs

In [ ]:
class PathConfig:

  def __init__(self, drive_dataset_path, pickle_save_path):
    self.DRIVE_DATASET_PATH = drive_dataset_path
    self.PICKLE_SAVE_PATH = pickle_save_path
    if not os.path.exists(pickle_save_path):
      os.mkdir(pickle_save_path)

  DRIVE_DATASET_PATH = "/content/data/MyDrive/Research/dataset"
  PICKLE_SAVE_PATH = "/content/data/MyDrive/Research/dataset/parquets"

  # preprocess paths
  TMP_UIDS_PATH = './tmp_uid.parquet'
  TMP_ITEMS_PATH = './tmp_itemid.parquet'

  paths = {
      'UID_MAP_PATH': 'uid_map.parquet',
      'ITEMID_MAP_PATH': './itemid_map.parquet',
      'RELATIONS_MAP_PATH': './relations_map.parquet',
      'TAGS_MAPPING_PATH': './tags_map.parquet',
      'BRANDS_MAPPING_PATH': './brands_map.parquet',
      'CATEGORY_MAPPING_PATH': './category_map.parquet',
      'CATEGORY_HIERARCHY_PATH': './category_hierarchy.parquet',
      'ITEMMETA_PARQUET_PATH': './itemmeta.parquet',
      'REVIEW_PARQUET_PATH': './review.parquet',
      'USER_EMBEDDINGS_PATH': './userembeddings.parquet',
      'ITEM_EMBEDDIGNS_PATH': './itemembeddings.parquet',
      'TRIPLES_ITEMCAT_PATH': './triples_itemcat.parquet',
      'TRIPLES_BRANDS_PATH': './triples_brand.parquet',
      'TRIPLES_TAGS_PATH': './triples_tags.parquet',


      'INDEXER_DATA_PATH': './indexer_path.pickle',
      'TFIDF_MODEL_PATH': './tfidf.pickle'
  }

  @property
  def UID_MAP_PATH(self):
    return self.paths['UID_MAP_PATH']

  @property
  def ITEMID_MAP_PATH(self):
    return self.paths['ITEMID_MAP_PATH']

  @property
  def RELATIONS_MAP_PATH(self):
    return self.paths['RELATIONS_MAP_PATH']

  @property
  def TAGS_MAPPING_PATH(self):
    return self.paths['TAGS_MAPPING_PATH']

  @property
  def ITEMMETA_PARQUET_PATH(self):
    return self.paths['ITEMMETA_PARQUET_PATH']

  @property
  def REVIEW_PARQUET_PATH(self):
    return self.paths['REVIEW_PARQUET_PATH']

  @property
  def BRANDS_MAPPING_PATH(self):
    return self.paths['BRANDS_MAPPING_PATH']

  @property
  def CATEGORY_MAPPING_PATH(self):
    return self.paths['CATEGORY_MAPPING_PATH']

  @property
  def CATEGORY_HIERARCHY_PATH(self):
    return self.paths['CATEGORY_HIERARCHY_PATH']

  @property
  def USER_EMBEDDINGS_PATH(self):
    return self.paths['USER_EMBEDDINGS_PATH']

  @property
  def ITEM_EMBEDDIGNS_PATH(self):
    return self.paths['ITEM_EMBEDDIGNS_PATH']

  @property
  def TRIPLES_ITEMCAT_PATH(self):
    return self.paths['TRIPLES_ITEMCAT_PATH']

  @property
  def TRIPLES_BRANDS_PATH(self):
    return self.paths['TRIPLES_BRANDS_PATH']

  @property
  def TRIPLES_TAGS_PATH(self):
    return self.paths['TRIPLES_TAGS_PATH']

  @property
  def TFIDF_MODEL_PATH(self):
    return self.paths['TFIDF_MODEL_PATH']

  @property
  def INDEXER_DATA_PATH(self):
    return self.paths['INDEXER_DATA_PATH']

  def copy_to_drive(self):
    self.use_local_paths()
    for path in self.paths:
      if os.path.exists(self.paths[path]):
        print('copy_to_drive: copying ', self.paths[path])
        shutil.copy(self.paths[path], os.path.join(self.DRIVE_DATASET_PATH, 'parquets'))

  def copy_to_local(self):
    self.use_drive_paths()
    for path in self.paths:
      if os.path.exists(self.paths[path]):
        shutil.copy(self.paths[path], './')
    self.use_local_paths()

  def use_drive_paths(self):
    for path in self.paths:
      self.paths[path] = os.path.join(self.DRIVE_DATASET_PATH, 'parquets', os.path.basename(self.paths[path]))

  def use_local_paths(self):
    for path in self.paths:
      self.paths[path] = './'+os.path.basename(self.paths[path])

In [ ]:
class Config:
  paths = PathConfig("/content/data/MyDrive/Research/dataset", "/content/data/MyDrive/Research/dataset/parquets")
  use_newdata = False
  use_bigdata = True
  use_parquets = False

In [ ]:
config = Config()

review_file = 'review.jsonl'
meta_file = 'meta.jsonl'

def init():

  if (not config.use_newdata) and (not config.use_bigdata) and (not config.use_parquets):
    return

  if config.use_parquets:
    config.paths.copy_to_local()
    return

  if config.use_bigdata:
    global review_file, meta_file
    review_file = 'cellphones_review.jsonl'
    meta_file = 'cellphones_meta.jsonl'

  if config.use_newdata:
    !rm ./{review_file}
    !rm ./{meta_file}

  if not os.path.exists('./' + review_file):
    !cp {config.paths.DRIVE_DATASET_PATH}/{review_file} ./
  if not os.path.exists('./' + meta_file):
    !cp {config.paths.DRIVE_DATASET_PATH}/{meta_file} ./

init()

In [ ]:
!rm *.parquet

In [ ]:
!cp {config.paths.DRIVE_DATASET_PATH}/parquets/itemmeta_v1.parquet ./itemmeta.parquet

# Test

# Dataset Pre-processing By Batches

In [ ]:
def save_pickle(obj, filename):
  with open(filename, 'w') as f:
    pickle.dump(obj, f)

In [ ]:
from collections.abc import Callable
import pyarrow as pa
import pyarrow.parquet as pq
import os

def write_parquet(outputParquetPath, df: pd.DataFrame, append=False):
  if isinstance(df, dd.DataFrame):
    df = df.compute()

  old_data = None
  if os.path.exists(outputParquetPath):
    if not append:
      os.remove(outputParquetPath)
    else:
      old_data = pq.read_table(outputParquetPath)

  table = pa.Table.from_pandas(df)
  if old_data:
    table = pa.concat_tables([old_data, table])

  pq.write_table(table, outputParquetPath, compression='snappy')


In [ ]:
title_tfidf_encoder = TfidfVectorizer(stop_words='english')
feature_tfidf_encoder = TfidfVectorizer(stop_words='english')
price_encoder = MinMaxScaler()
rating_encoder = MinMaxScaler()

In [ ]:
from datetime import datetime

def sinusoidal_transform(timestamp):
  dt = datetime.fromtimestamp(timestamp)
  return [
      np.sin(2 * np.pi * dt.day/31),
      np.cos(2 * np.pi * dt.day/31),
      np.sin(2 * np.pi * dt.month/12),
      np.cos(2 * np.pi * dt.month/12)
  ]

def time_encoder(ddf: dd.DataFrame) -> dd.Series:
  ddf = ddf.assign(
      sin_day= ddf['unixReviewTime'].apply(lambda x: sinusoidal_transform(x)[0]),
      cos_day= ddf['unixReviewTime'].apply(lambda x: sinusoidal_transform(x)[1]),
      sin_month = ddf['unixReviewTime'].apply(lambda x: sinusoidal_transform(x)[2]),
      cos_month= ddf['unixReviewTime'].apply(lambda x: sinusoidal_transform(x)[3])
  )
  return ddf

def time_decoder(ddf: dd.DataFrame) -> dd.DataFrame:
  timestamp = (31/(2*np.pi)) * np.arctan2(ddf['sin_day'], ddf['cos_day'])
  ddf = ddf.assign(timestamp=timestamp)

In [ ]:
from collections.abc import Callable

def batch_extract(file, extractor: Callable[[pd.DataFrame], pd.DataFrame]):
  chunk_size = 20000

  with open(file, 'r') as file:
      batch = []
      for i, line in enumerate(file):
          try:
            batch.append(json.loads(line))
          except Exception as e:
            print(f"Error processing line {i}: {e}", line)
            continue

          # batch loaded therefore processing calling extractor...
          if (i + 1) % chunk_size == 0:
              df = pd.DataFrame(batch)
              ddf = dd.from_pandas(df, 2)
              df = extractor(ddf)

              print(f"Processed batch {(i + 1) // chunk_size}")
              batch = []  # Clear the batch for the next chunk

      # Process the last batch. if there any remaining
      if batch:
          df = pd.DataFrame(batch)
          ddf = dd.from_pandas(df)
          df = extractor(ddf)
          print(f"Processed final batch")

In [ ]:
def extract_review_features(ddf: dd.DataFrame):
  # Following features are not used : style, reviewText, summary, image
  ddf_final = ddf[['reviewerID', 'asin', 'overall', 'unixReviewTime']]
  ddf_final = ddf_final.map_partitions(time_encoder, meta={
                                    **ddf_final.dtypes,
                                    'sin_day': 'f8',
                                    'cos_day': 'f8',
                                    'sin_month': 'f8',
                                    'cos_month': 'f8',
                                  })
  write_parquet(config.paths.REVIEW_PARQUET_PATH, ddf_final, append=True)

  return ddf_final

In [ ]:
config.paths.copy_to_drive()

In [ ]:
batch_extract(review_file, extract_review_features)

In [ ]:
def extract_itemmeta_features(ddf):
  # we remove ['tech1', ]
  batchDf = ddf[['asin', 'title', 'price', 'brand', 'category', 'also_buy', 'rank', 'feature']]
  write_parquet(config.paths.ITEMMETA_PARQUET_PATH, batchDf, append=True)
  return batchDf

In [ ]:
batch_extract(meta_file, extract_itemmeta_features)

In [ ]:
fc = pd.Series([cat for row in df['category'] for cat in row], name='a').value_counts()
fc = fc[fc > 0]
fc[-20:]

In [ ]:
config.paths.use_local_paths()

In [ ]:
# @title
import re
import spacy
import joblib

nlp = spacy.load('en_core_web_sm')

def last_cat_helper_(df, flat_cats):
  s = ''
  truncated_cats=[]

  for a in df.category:
    if a not in flat_cats.index:
      break
    else:
      truncated_cats.append(a)
      s = a
  return [s, truncated_cats]

def extract_last_cat(df):
  df.category = df.category.apply(lambda x:  ast.literal_eval(x))
  flat_cats = pd.Series([cat for row in df['category'] for cat in row], name='a').value_counts()
  flat_cats = flat_cats[flat_cats > 4]
  last_cats = df.apply(last_cat_helper_, args=(flat_cats,), axis=1, result_type='expand')
  df.loc[:, 'category'] = last_cats[1]
  df.loc[:, 'cat'] = last_cats[0]
  return df


def cat_helper(row):
  if row['cat'] in row['rank']:
    row['rank'] = row['rank'].replace("'", '"')
    current_rank = row['rank']
    try:
      rank_cats = json.loads(current_rank)
    except Exception as e:
      print(e, current_rank)

    for a in rank_cats:
      pattern = re.compile("'?>?#?([\d,]+)\s.*"+re.escape(row['cat']))
      m = pattern.search(a)
      if m:
        rank = int(m.group(1).replace(",", ""))
        return rank
  else:
    return np.nan


def extract_cat_rank(df):
  #display(ddf[['rank', 'cat']].compute())
  print(df.columns)
  df[:, 'cat_rank'] = df.apply(cat_helper, axis=1)
  return df

def get_tokens(text) -> set:
  text = html.unescape(text)
  text = re.sub(r'[^a-zA-Z0-9 &]+', '', text)
  doc = nlp(text)
  tokens = set()
  remove_set = set(['official', 'equipment', 'original', 'oem', "heavy", "duty", "portable", "fast", "universal", "premium", "compact"])

  for a in doc:
    if a.pos_ == "NUM" and a.head.pos_ in ["PROPN", "NOUN", "ADP"] and a.head.dep_ in ['ROOT', 'pobj', 'nmod', 'nsubj', 'conj']:
      tokens.add(a.head.text + " " + a.text)
      continue
    elif a.dep_ in ["compound", "pobj", "ROOT", "nmod"]  and a.head.dep_ in ["ROOT", "pobj", "nmod", 'nsubj', 'compound', 'prep']:
      tokens.add(a.lemma_.lower())

  tokens = tokens - remove_set
  return tokens

rtokens = set()

def apply_nlp_ner(row):
  tokens = get_tokens(row)
  for a in tokens:
    rtokens.add(a)
  return [*tokens]

def extract_tags(df: pd.DataFrame) -> pd.DataFrame:
  print(df)
  # nlp = spacy.load('en_core_web_sm')
  tags = df['title'].apply(apply_nlp_ner)
  df['tags'] = tags
  print(df, type(df))
  # print('docs', docs)
  return df


def extract_title_tfidf(df: pd.DataFrame) -> dd.DataFrame:
  vectorizer = TfidfVectorizer(max_features=32)
  X = vectorizer.fit_transform(df['title'])
  n = np.array([json.dumps(vec.tolist()) for vec in X.toarray()])
  df['title_tfidf'] = n
  df.drop(columns=['title'], inplace=True)
  joblib.dump(vectorizer, config.paths.TFIDF_MODEL_PATH)
  return df


def build_item_features():
  df = pd.read_parquet(config.paths.ITEMMETA_PARQUET_PATH)
  df = extract_last_cat(df)
  write_parquet('./1cats.parquet', df)
  df = extract_tags(df)
  write_parquet('./2tags.parquet', df)
  df = extract_title_tfidf(df)

  write_parquet(config.paths.ITEMMETA_PARQUET_PATH, df)


build_item_features()


In [ ]:
df = pd.read_parquet(config.paths.ITEMMETA_PARQUET_PATH, columns=['title'])
df

In [ ]:
config.paths.use_local_paths()

In [ ]:
del df
del config

In [ ]:
config = Config()
df = pd.read_parquet(config.paths.ITEMMETA_PARQUET_PATH, columns=[ 'category'])

## current

In [ ]:
class Indexer:
  data = {}
  to_graph = {}
  from_graph = {}
  rel_types = {}
  reverse_rel_types = {}
  counter = 0
  config: Config = None

  def __init__(self, config):
    self.counter = 0
    self.config = config
    pass

  def persist(self):
    pickler = IndexerPickler(self)
    pickler.persist(self.config.paths.INDEXER_DATA_PATH)

  @staticmethod
  def load(config):
    indexer = IndexerPickler.load(config.paths.INDEXER_DATA_PATH, config)
    return indexer


  def add_rel(self, id, reltype: str):
    self.rel_types[reltype] = id
    self.reverse_rel_types[id] = reltype


  def add_node(self, local_id, rel_type):
    self.to_graph[local_id] = self.counter
    self.from_graph[self.counter] = local_id
    if rel_type in self.data:
      self.data[rel_type].add(self.counter)
    else:
      self.data[rel_type] = {self.counter}
    self.counter += 1


  def add_nodes(self, local_ids, rel_type):
    for a in range(len(local_ids)):
      self.to_graph[a] = self.counter
      self.from_graph[self.counter] = a
      if rel_type in self.data:
        self.data[rel_type].add(self.counter)
      else:
        self.data[rel_type] = {self.counter}
      self.counter += 1

  def get_rel_type_single_(self, graph_id) -> str:
    for key in self.data:
      if graph_id in self.data[key]:
        return key

  def reltype_to_index(self, reltypes: list | str) -> list | int:
    if isinstance(reltypes, str):
      return self.rel_types[reltypes]
    else:
      index_list = []
      for a in reltypes:
        index_list.append(self.rel_types[a])
      return index_list


  def index_to_reltype(self, index: list | str) -> list | str:
    if isinstance(index, int):
      return self.reverse_rel_types[index]
    else:
      rel_list = []
      for a in index:
        rel_list.append(self.reverse_rel_types[a])
      return rel_list


  def rt(self, graph_ids) -> str | list:
    if not isinstance(graph_ids, list):
      return self.get_rel_type_single_(graph_ids)

    rel_types = []
    for a in graph_ids:
      rel_types.append(self.get_rel_type_single_(a))
    return rel_types


  def gi(self, local_index):
    '''Give the graph index providex the local index'''
    if isinstance(local_index, list):
      indexes = []
      for a in local_index:
        indexes.append(self.to_graph[a])
      return indexes
    return self.to_graph[local_index]

  def li(self, graph_Index):
    '''Give the local index prodvided graph index (or indexes)'''
    if isinstance(graph_Index, list):
      indexes = []
      for a in graph_Index:
        indexes.append(self.from_graph[a])
      return indexes
    return self.from_graph[graph_Index]



In [ ]:
class IndexerPickler:
  data = {}
  to_graph = {}
  from_graph = {}
  rel_types = {}
  reverse_rel_types = {}
  counter = 0

  def __init__(self, index:Indexer):
    self.counter =  index.counter
    self.data =  index.data
    self.to_graph =  index.to_graph
    self.from_graph =  index.from_graph
    self.rel_types =  index.rel_types
    self.reverse_rel_types =  index.reverse_rel_types
    self.counter = index.counter

  def persist(self, path):
    with open(path, 'wb') as f:
      pickle.dump(self, f)

  @staticmethod
  def load(path, config):
    indexer = None
    with open(path, 'rb') as f:
      ctx = pickle.load(f)
      indexer = Indexer(config)
      indexer.counter = ctx.counter
      indexer.data = ctx.data
      indexer.to_graph = ctx.to_graph
      indexer.from_graph = ctx.from_graph
      indexer.rel_types = ctx.rel_types
      indexer.reverse_rel_types = ctx.reverse_rel_types
      return indexer

In [ ]:
import gc
gc.collect()

In [ ]:
import gc
import ast

def safe_jsonload(string:str):
  '''For debugging some items are not parsed and exception thrown.'''
  try:
    item = json.loads(string)
    return item
  except json.JSONDecodeError as e:
    print('Decoding ', string, 'caused : ', e)
    count, idx = 0, 0
    for i in range(len(temp)):
      if temp[i] == '"':
        count += 1
      if count == 4:
        idx = i
        break
    return json.loads(temp[:idx+1]+ ']')

def extract_categories(items):
  categories = []
  pattern = re.compile(r'^[a-zA-Z &]+$')

  for item in items:
    words = item.split()
    # Rule 1: Most words are capitalized
    if sum(word[0].isupper() for word in words) / len(words) > 0.7:
      # Rule 2: Length should be reasonable for a category (2-5 words)
      if 1 <= len(words) <= 5:
        if pattern.match(item):
          categories.append(item)
  return categories

class DataTransformer:
  review_df = None
  meta_df = None
  data = {}

  def __init__(self, config: Config, indexer: Indexer):
    self.config = config
    self.indexer = indexer;
    relation_types = [
      'purchase',
      'catcat',  # category hierarchial relations
      'catuser', # user-category
      'catitem', # item-category
      'tag', # item-tag
      'brand' # item-brand
    ]
    relations_df = pd.DataFrame({'id': np.arange(1, len(relation_types)+1) ,"rel": relation_types})
    write_parquet(config.paths.RELATIONS_MAP_PATH, relations_df)


  def load_relations(self):
    rel_df = self.get_parquet_data(self.config.paths.RELATIONS_MAP_PATH)
    for idx, row in rel_df.iterrows():
      self.indexer.add_rel(row['id'], row['rel'])


  def delete_review_data(self):
    del self.review_df
    gc.collect()


  def delete_meta_data(self):
    del self.meta_df
    gc.collect()


  def get_review_data(self) -> pd.DataFrame:
    if self.review_df:
      return self.review_df
    else:
      self.review_df = dd.read_parquet(self.config.paths.REVIEW_PARQUET_PATH).compute()
      return self.review_df

  def get_meta_data(self) -> pd.DataFrame:
    if self.meta_df:
      return self.meta_df
    else:
      self.meta_df = dd.read_parquet(self.config.paths.ITEMMETA_PARQUET_PATH).compute()
      return self.meta_df


  def get_parquet_data(self, path) -> pd.DataFrame:
    if path in self.data:
      return self.data[path]
    else:
      self.data[path] = dd.read_parquet(path).compute()
      return self.data[path]


  def extract_map(self, df, mappingCol, outputParquetPath, renameCol=None):
    print(mappingCol)
    map_df = df[[mappingCol]].drop_duplicates(ignore_index=True)
    map_df['id'] = np.arange(len(map_df))
    map_df = map_df[['id', mappingCol]]
    if renameCol:
      map_df = map_df.rename(columns=dict((mappingCol, renameCol)))
    write_parquet(outputParquetPath, map_df)
    self.indexer.add_nodes(map_df['id'], mappingCol)
    return map_df


  def assign_unique_ids_util_(self, df, col, uniqueDf):
    df[col] = pd.merge(df, uniqueDf, on=col, how='left').id


  def build_category_heirarchies(self, categories: pd.Series, uniqueCats: pd.DataFrame):
    cat_heirarchies = set()
    for cats in categories:
      if cats is not None and len(cats) > 0:
        for i in range(len(cats)-1):
          a1 = uniqueCats[uniqueCats['cat'] == cats[i]].iloc[0].id
          #a2 = uniqueCats[uniqueCats['cat'] == cats[i+1]].iloc[0].id
          temp = uniqueCats[uniqueCats['cat'] == cats[i+1]]
          if temp.empty:
            print('cat is ', cats[i+1])
            raise Exception()
          else:
            a2 = temp.iloc[0].id
            cat_heirarchies.add((a1, a2))

    subjects = [item[0] for item in cat_heirarchies]
    objects  = [item[1] for item in cat_heirarchies]
    heirarchy_ds = pd.DataFrame(data={"parent": subjects, "child": objects})
    write_parquet(self.config.paths.CATEGORY_HIERARCHY_PATH, heirarchy_ds)
    return heirarchy_ds


  def build_mappings(self):
    meta_df = self.get_meta_data()

    unique_asins = self.extract_map(meta_df, 'asin', self.config.paths.ITEMID_MAP_PATH)
    self.assign_unique_ids_util_(meta_df, 'asin', unique_asins)

    review_df = self.get_review_data()

    review_df = review_df.rename(columns={'reviewerID': 'uid'})
    self.assign_unique_ids_util_(review_df, 'asin', unique_asins)

    unique_reviewers = self.extract_map(review_df, 'uid', self.config.paths.UID_MAP_PATH)
    self.assign_unique_ids_util_(review_df, 'uid', unique_reviewers)

    unique_brands = self.extract_map(meta_df, 'brand', self.config.paths.BRANDS_MAPPING_PATH)
    self.assign_unique_ids_util_(meta_df, 'brand', unique_brands)

    #rating normalization
    rating_encoder = MinMaxScaler()
    review_df['rating'] = rating_encoder.fit_transform(review_df[['overall']])

    # category mapping and heirarchy building
    meta_df['category'] = meta_df['category'].apply(lambda x: extract_categories(x))
    print(meta_df['category'])
    # temp3 = pd.Series([cat for row in temp2 for cat in row], name='a').value_counts()
    # temp3 = temp3[temp3 > 4]
    flat_list = pd.Series([cat for row in meta_df['category'] for cat in row]).value_counts()
    unique_cats = set(flat_list.index.tolist())
    mapping_cat_df = pd.DataFrame(data={'id': np.arange(len(unique_cats)), 'cat': [*unique_cats]})
    print(unique_cats)
    self.build_category_heirarchies(meta_df['category'], mapping_cat_df)
    self.assign_unique_ids_util_(meta_df, 'cat', mapping_cat_df)
    self.indexer.add_nodes(mapping_cat_df['id'], 'cat')
    meta_df.fillna(mapping_cat_df.loc[mapping_cat_df['cat'] == flat_list.index[0]].iloc[0].id, inplace=True)

    # Tag mapping
    tag_flat_list = [tag for row in meta_df['tags'] for tag in row]
    unique_tags = set(tag_flat_list)
    mapping_tag_df = pd.DataFrame(data={'id': np.arange(len(unique_tags)), 'tag': [*unique_tags]})
    self.indexer.add_nodes(mapping_tag_df['id'], 'tag')

    meta_df.drop(columns=['price', 'category', 'also_buy', 'rank', 'feature'], inplace=True)

    write_parquet(self.config.paths.TAGS_MAPPING_PATH, mapping_tag_df)
    write_parquet(self.config.paths.CATEGORY_MAPPING_PATH, mapping_cat_df)
    write_parquet(self.config.paths.ITEM_EMBEDDIGNS_PATH, meta_df)
    write_parquet(self.config.paths.USER_EMBEDDINGS_PATH, review_df)

    self.indexer.persist()


  def build_triples(self):
    itememb = self.get_parquet_data(self.config.paths.ITEM_EMBEDDIGNS_PATH)

    brand_triples = itememb[['asin', 'brand']]
    write_parquet(self.config.paths.TRIPLES_BRANDS_PATH, brand_triples)

    itemcat_triples = itememb[['asin', 'cat']]
    write_parquet(self.config.paths.TRIPLES_ITEMCAT_PATH, itemcat_triples)

    tags = itememb[['asin', 'tags']].explode('tags', ignore_index=True)
    tagsmapping = self.get_parquet_data(self.config.paths.TAGS_MAPPING_PATH)
    tag_groups = tagsmapping.groupby('tag')['id'].first()
    tags['tags'] = [tag_groups.get(tag) for tag in tags['tags']]
    write_parquet(self.config.paths.TRIPLES_TAGS_PATH, tags)

  def build_embeddings(self):
    # Item embeddings
    itememb = self.get_parquet_data(self.config.paths.ITEM_EMBEDDIGNS_PATH)
    itememb = itememb.join(itememb['title_tfidf']
                 .apply(lambda x: json.loads(x))
                 .apply(pd.Series).rename(columns= lambda x: 'tf_' + str(x)))
    # cat_rank was removed. reminder: add it back if added above.
    itememb = itememb.drop(columns=['brand', 'cat', 'tags', 'title_tfidf'])
    write_parquet(self.config.paths.ITEM_EMBEDDIGNS_PATH, itememb)

    # Purchase relation embeddings
    purchase_emb = self.get_parquet_data(self.config.paths.USER_EMBEDDINGS_PATH)
    purchase_emb = purchase_emb.drop(columns=['overall'])
    write_parquet(self.config.paths.USER_EMBEDDINGS_PATH, purchase_emb)

  def clear_parquets(self):
    '''Warning: for manual calling only. will delete parquets. util'''
    review_df = None
    meta_df = None
    data = {}
    !rm ./brands_map.parquet
    !rm itemid_map.parquet
    !rm relations_map.parquet
    !rm uid_map.parquet
    !rm category_hierarchy.parquet
    !rm category_map.parquet
    !rm indexer_path.pickle
    !rm itemembeddings.parquet
    !rm tags_map.parquet
    !rm triples_itemcat.parquet
    !rm triples_brands.parquet
    !rm triples_tags.parquet
    !rm userembeddings.parquet
    !rm triples*.parquet
    gc.collect()


  def transform(self):
    self.load_relations()
    self.build_mappings()
    self.build_triples()
    self.build_embeddings()




In [ ]:
# if config != None:
#   del config
# if g_indexer != None:
#   del g_indexer
# if transformer != None:
#   del transformer
config = Config()
config.paths.use_local_paths()
g_indexer = Indexer(config)
transformer = DataTransformer(config, g_indexer)
transformer.transform()
# transformer.clear_parquets()

In [ ]:
!cp {config.paths.DRIVE_DATASET_PATH}/parquets/review.parquet ./review.parquet

In [ ]:
df = pd.read_parquet(config.paths.ITEMMETA_PARQUET_PATH, columns=['category', 'cat'])
temp = df.cat.value_counts()

In [ ]:
!ls -la *.parquet

In [ ]:
config.paths.copy_to_drive()

In [ ]:
!cp ./2tags.parquet ./data/MyDrive/

## Fn

## Recommendation Graph Construction

In [ ]:
import networkx as nx

In [ ]:
knowledge_graph = nx.Graph()

####  Add user-item interactions edges

In [ ]:
for idx, row in review_data.iterrows():
  knowledge_graph.add_edge(f"u_{row['reviewerID_encoded']}", f"i_{row['asin_encoded']}",
               type='interacts', weight=row['helpful_votes'])

## GNN implementation

In [ ]:
!pip install torch-geometric

In [ ]:
import torch as t
import torch.nn as nn
import torch_geometric.nn as pyg_nn

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, num_users, num_items, emb_dim):
        super(GNNModel, self).__init__()
        self.embedding_dim = emb_dim

        self.user_embeddings = nn.Embedding(num_users, emb_dim)
        self.item_embeddings = nn.Embedding(num_items, emb_dim)

        self.conv1 = pyg_nn.GCNConv(emb_dim, emb_dim)
        self.conv2 = pyg_nn.GCNConv(emb_dim, emb_dim)

    def forward(self, data):

        x = t.cat([self.user_embeddings.weight, self.item_embeddings.weight], dim=0)

        # GCN Layer 1
        x = self.conv1(x, data.edge_index, edge_attr=data.edge_attr)
        x = t.relu(x)

        # GCN Layer 2
        x = self.conv2(x, data.edge_index, edge_attr=data.edge_attr)
        return x

In [ ]:
class DotPredictor(pyg_nn.MessagePassing):
    def forward(self, data, h):
        x = data.x
        edge_index = data.edge_index


In [ ]:
import itertools

In [ ]:
with open(m) as f:
  for i in range(1000):
    val = f.readline()
    obj = json.loads(val)
    features = obj['rank']
    a2 = obj['category']
    #for feature in features:
    if features:
      print('[', features,',', a2, ']')

In [ ]:
s = [ ['>#6,681,833 in Cell Phones & Accessories (See top 100)', '>#286,289 in Cell Phones & Accessories > Cell Phone Accessories > Accessory Kits', '>#6,677,630 in Electronics > Cell Phones & Accessories'] , ['Cell Phones & Accessories', 'Accessories', 'Accessory Kits'] ]

for a in s:
  for b in a:
    print(b)

In [ ]:
jsons = []

with open(m, 'r') as f:
  for a in range(500):
    if a < 200:
      continue
    jsons.append(f.readline())

df = pd.read_json('\n'.join(jsons), lines=True)
df.to_csv('cellphones_meta.csv')

In [ ]:
def get_pd(file, take, skip):
  jsons = []

  with open(file, 'r') as f:
    for a in range(take):
      if a < skip:
        continue
      jsons.append(f.readline())

  df = pd.read_json('\n'.join(jsons), lines=True)
  df.to_csv(file + '.csv')
  return df

In [ ]:
def download_to_csv(url, jsonName, skip, take):
  if not os.path.exists(jsonName):
    saved_result = urllib.request.urlretrieve(url, jsonName)
    print(saved_result)
    extract_gz(saved_result[0], jsonName)

  jsons = []

  with open(jsonName, 'r') as f:
    for a in range(take):
      if a < skip:
        continue
      jsons.append(f.readline())

  df = pd.read_json('\n'.join(jsons), lines=True)
  df.to_csv(jsonName + '.csv')

In [ ]:
download_to_csv('https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Tools_and_Home_Improvement_5.json.gz',
                'movies.json',
                200,
                500)

In [ ]:
!head movies.json

In [ ]:
!curl -k -o office_products.json.gz https://mcauleylab.ucsd.edu/public_datasets/data/amazon_v2/metaFiles2/meta_Office_Products.json.gz

In [ ]:
!gunzip -r ./office_products.json.gz

In [ ]:
df = pd.read_json('./afn', lines=True)

In [ ]:
df[df['reviewerID'] == 'A3H9DSJS96YY84']

In [ ]:
get_pd('office_products.json', 500, 100)

In [ ]:
df = pd.read_csv('./office_products.csv')

In [ ]:
features = df.tech1

for a in range(10):
  print(features[a])

In [ ]:
import ast
import itertools

strs = set()

for feature_str in features:
  try:
    feature = ast.literal_eval(feature_str)
  except Exception as e:
    print(e)
    break
  if feature and len(feature) > 0:
    for a in feature:
      strs.add(a)
for a in strs:
  print(a)